In [1]:
###Cargando los paquetes 
using JuMP, HiGHS, LinearAlgebra, Plots, Random, CSV, DataFrames, XLSX

In [2]:
#Cargamos los datos 
G=XLSX.readtable("DietProblemPolly.xlsx","Data")
### Tipo de G
println("El tipo de G es\n",typeof(G),"\n")
### G tiene dos componentes: G.data y G.column_labels
println("G.data es\n",G.data,"\n")
println("G.column_labels es\n",G.column_labels)

El tipo de G es
XLSX.DataTable



G.data es
Any[

Any[

"Oatmeal", "Chicken ", "Eggs", "Whole milk ", "Cherry pie ", "Pork with beans ", "Minimo "], Any[4.0, 32.0, 13.0, 8.0, 4.0, 14.0, 55.0], Any[2.0, 12.0, 54.0, 285.0, 22.0, 80.0, 800.0], Any[3.0, 24.0, 13.0, 9.0, 20.0, 19.0, 

missing], Any[4.0, 3.0, 2.0, 8.0, 2.0, 2.0, missing]]



G.column_labels es


[:Food, Symbol("Protein "), Symbol("Calcium "), Symbol("Price per seving "), Symbol("Limit per day ")]


In [3]:
data = DataFrame(G);
display(data)

Row,Food,Protein,Calcium,Price per seving,Limit per day
,Any,Any,Any,Any,Any
1,Oatmeal,4.0,2.0,3.0,4.0
2,Chicken,32.0,12.0,24.0,3.0
3,Eggs,13.0,54.0,13.0,2.0
4,Whole milk,8.0,285.0,9.0,8.0
5,Cherry pie,4.0,22.0,20.0,2.0
6,Pork with beans,14.0,80.0,19.0,2.0
7,Minimo,55.0,800.0,missing,missing


In [15]:
###Creando el modelo en conjunto con el solver a usar
modelo = Model(HiGHS.Optimizer) 
println(modelo)

n=6
m=3
B=3


Feasibility
Subject to



3

In [16]:
#creamos variables para ver cuantas porciones de cada comida son
@variable(modelo, x[1:6]>=0,Int) #esto crea el arreglo [x[1] ,...,x[6]]
#creamos unas variables binarias para ver si la comida i se come o no 
@variable(modelo, y[1:6], Bin) 
#agregamos las restricciones de nutrientes
@constraint(modelo, protein[j in 2:3], sum(x[i]*data[i,j] for i in 1:6) >= data[7,j])
#agregamos las restricciones de limite de servings per day
@constraint(modelo, limit[i in 1:6],x[i]<=data[i,5])
#restriccion big-M para ver si comida i se come o no 
@constraint(modelo, Secome[i in 1:6],x[i]>=0-B*(1-y[i]))
#restriccion big-M para ver si comida i se come o no 
@constraint(modelo, NoSeCome[i in 1:6],x[i]<=B*y[i])
#restriccion de que no se pueden comprar mas de B comidas 
@constraint(modelo, limiteComidas,sum(y[i] for i in 1:6)<=B)


limiteComidas : y[1] + y[2] + y[3] + y[4] + y[5] + y[6] <= 3.0

In [17]:
@objective(modelo, Min, sum(x[i]*data[i,4] for i in 1:6))
println(modelo)

Min 3 x[1] + 24 x[2] + 13 x[3] + 9 x[4] + 20 x[5] + 19 x[6]
Subject to
 protein[2] : 4 x[1] + 32 x[2] + 13 x[3] + 8 x[4] + 4 x[5] + 14 x[6] >= 55.0
 protein[3] : 2 x[1] + 12 x[2] + 54 x[3] + 285 x[4] + 22 x[5] + 80 x[6] >= 800.0
 Secome[1] : x[1] - 3 y[1] >= -3.0
 Secome[2] : x[2] - 3 y[2] >= -3.0
 Secome[3] : x[3] - 3 y[3] >= -3.0
 Secome[4] : x[4] - 3 y[4] >= -3.0
 Secome[5] : x[5] - 3 y[5] >= -3.0
 Secome[6] : x[6] - 3 y[6] >= -3.0
 limit[1] : x[1] <= 4.0
 limit[2] : x[2] <= 3.0
 limit[3] : x[3] <= 2.0
 limit[4] : x[4] <= 8.0
 limit[5] : x[5] <= 2.0
 limit[6] : x[6] <= 2.0
 NoSeCome[1] : x[1] - 3 y[1] <= 0.0
 NoSeCome[2] : x[2] - 3 y[2] <= 0.0
 NoSeCome[3] : x[3] - 3 y[3] <= 0.0
 NoSeCome[4] : x[4] - 3 y[4] <= 0.0
 NoSeCome[5] : x[5] - 3 y[5] <= 0.0
 NoSeCome[6] : x[6] - 3 y[6] <= 0.0
 limiteComidas : y[1] + y[2] + y[3] + y[4] + y[5] + y[6] <= 3.0
 x[1] >= 0.0
 x[2] >= 0.0
 x[3] >= 0.0
 x[4] >= 0.0
 x[5] >= 0.0
 x[6] >= 0.0
 x[1] integer
 x[2] integer
 x[3] integer
 x[4] integer
 x[

In [18]:
# Resolver el problema de optimización con HiGHS
optimize!(modelo)

println("Optimal value: ", objective_value(modelo))

Running HiGHS 1.4.2 [date: 1970-01-01, git hash: f797c1ab6]
Copyright (c) 2022 ERGO-Code under MIT licence terms
Presolving model
9 rows, 12 cols, 30 nonzeros
8 rows, 11 cols, 27 nonzeros
Objective function is integral with scale 1

Solving MIP model with:
   8 rows
   11 cols (6 binary, 5 integer, 0 implied int., 0 continuous)
   27 nonzeros

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
     Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   18              inf                  inf        0      0      0         0     0.0s
 R       0       0         0   0.00%   50.07654185     60                16.54%        0      0      0         4     0.0s

18.2% inactive integer columns, restarting
Model after restart has 6 rows, 8 cols (6 bin., 2 int., 0 impl., 0 cont.), and 16 nonzeros

         0       0         0 

In [19]:
for i in 1:6
    println("El valor de $(x[i]) es ",value(x[i]))

end

El valor de x[1] es -0.0
El valor de x[2] es 1.0
El valor de x[3] es 0.0
El valor de x[4] es 3.0
El valor de x[5] es 0.0
El valor de x[6] es 0.0


In [20]:
for i in 1:6
    println("El valor de $(y[i]) es ",value(y[i]))

end

El valor de y[1] es -0.0
El valor de y[2] es 1.0
El valor de y[3] es -0.0
El valor de y[4] es 1.0
El valor de y[5] es 0.0
El valor de y[6] es 0.0
